# Análisis e Inserción de Emisiones de CO₂ por País

Este notebook:  
1. Carga el CSV de emisiones de CO₂.  
2. Explora y limpia los datos (identifica valores faltantes).  
3. Normaliza nombres de país y gestiona excepciones para mapearlos contra la tabla `Paises`.  
4. Conecta a MySQL, añade la unidad “mt” a `Unidades` y crea la tabla `EmisionesCO2`.  
5. Inserta los registros anuales de emisiones en la tabla de hechos.


In [6]:
import os
import pandas as pd
import pymysql
from pymysql.constants import CLIENT
from pymysql import OperationalError

# (Opcional) Si usas un .env:
# from dotenv import load_dotenv
# load_dotenv()

DB_HOST     = os.getenv('DB_HOST', 'localhost')
DB_USER     = os.getenv('DB_USER', 'root')
DB_PASSWORD = os.getenv('DB_PASSWORD', 'admin')
DB_NAME     = os.getenv('DB_NAME', 'tfm_cambio_climatico')

# Conexión a MySQL
conexion = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME,
    client_flag=CLIENT.MULTI_STATEMENTS
)
cursor = conexion.cursor()


## 1. Carga y Exploración Inicial

Leemos el CSV y vemos su forma, columnas y primeros registros.


In [7]:

df_co2 = pd.read_csv('./data/tidy_format_co2_emission_dataset.csv', sep=',', encoding='utf-8')

# Vista rápida
print("Shape:", df_co2.shape)
display(df_co2.head())

# Comprobar missing values
print("\nValores nulos por columna:")
print(df_co2.isnull().sum())


Shape: (5572, 3)


,Country,Year,CO2EmissionRate (mt)
0,Afghanistan,2021,8.35
1,Albania,2021,4.59
2,Algeria,2021,173
3,Angola,2021,24.45
4,Antigua and Barbuda,2021,0.78



Valores nulos por columna:
Country                 0
Year                    0
CO2EmissionRate (mt)    0
dtype: int64


## 2. Limpieza y Normalización de Países

Creamos `country_norm`, aplicamos excepciones para casos irregulares, y mapeamos contra la dimensión `Paises`.


In [8]:
# ————————————————————————————————————————————————
# Celda X: Normalizar, excepciones, explode y listar no mapeados
# ————————————————————————————————————————————————

# 1) Normalizar country y CO2EmissionRate (mt)
df_co2['country_norm'] = (
    df_co2['Country']
      .astype(str)
      .str.strip()
      .str.lower()
)

# Quitar separadores de miles
df_co2['co2_mt'] = df_co2['CO2EmissionRate (mt)'] \
    .astype(str) \
    .str.replace(',', '', regex=False)

# Convertir a numérico, valores no válidos → NaN
df_co2['co2_mt'] = pd.to_numeric(df_co2['co2_mt'], errors='coerce')

# Informar y descartar filas con NaN en co2_mt
missing = df_co2['co2_mt'].isna().sum()
print(f"⚠️ Se encontraron {missing} filas con valores no numéricos en CO2EmissionRate y se descartarán.")
df_co2 = df_co2[df_co2['co2_mt'].notna()]

# 2) Cargar dimensión Paises (codigo, nombre_en)
cursor.execute("SELECT codigo, nombre_en FROM Paises;")
dim_paises = { ne.strip().lower(): code for code, ne in cursor.fetchall() }

# 3) Excepciones especiales
exceptions_co2 = {
    'bolivia': 'bolivia (plurinational state of)',
    'british virgin islands': 'virgin islands (british)',
    'brunei':'brunei darussalam',
    'cape verde': 'cabo verde',
    'czech republic': 'czechia',
    'democratic republic of the congo': 'congo (the democratic republic of the)',
    'falkland islands':    'falkland islands (the) [malvinas]',
    'iran': 'iran (islamic republic of)',
    'ivory coast': 'côte d\'ivoire',
    'laos': 'lao people\'s democratic republic',
    'macau': 'macao',
    'moldova': 'moldova (the republic of)',
    'namibia': None,
    'netherlands': 'netherlands (kingdom of the)',
    'north korea': 'korea (the democratic people\'s republic of)',
    'russia': 'russian federation',
    'são tomé and príncipe': 'sao tome and principe',
    'south korea': 'korea (the republic of)',
    'syria': 'syrian arab republic',
    'taiwan': 'taiwan (province of china)',
    'tanzania': 'tanzania, the united republic of',
    'turkey': 'türkiye',
    'united kingdom': 'united kingdom of great britain and northern ireland',
    'united states': 'united states of america',
    'venezuela': 'venezuela (bolivarian republic of)',
    'vietnam': 'viet nam'
}

# 4) Aplicar excepciones
df_co2['country_db_name'] = df_co2['country_norm'].map(
    lambda x: exceptions_co2[x] if x in exceptions_co2 else x
)

# 5) Explode de listas
df_co2['country_list'] = df_co2['country_db_name'].apply(
    lambda v: [] if v is None else (v if isinstance(v, list) else [v])
)
df_co2 = df_co2.explode('country_list')

# 6) Mapear a código
df_co2['pais_id'] = (
    df_co2['country_list']
      .str.strip()
      .str.lower()
      .map(dim_paises)
)

# 7) Filtrar sin mapeo
antes = len(df_co2)
df_co2 = df_co2[df_co2['pais_id'].notna()]
print(f"⚠️Se descartaron {antes - len(df_co2)} filas sin país válido")


⚠️ Se encontraron 122 filas con valores no numéricos en CO2EmissionRate y se descartarán.
⚠️Se descartaron 28 filas sin país válido


## 4. Crear la tabla `EmisionesCO2` y añadir la unidad “mt”

En esta sección creamos la tabla de hechos para las emisiones de CO₂ y nos aseguramos de que la unidad `mt` (metric tonnes) esté presente en la dimensión `Unidades`.


In [9]:
conexion.ping(reconnect=True)
cursor = conexion.cursor()

# Crear la tabla EmisionesCO2
sql_create_emisiones = """
CREATE TABLE IF NOT EXISTS EmisionesCO2 (
    id INT AUTO_INCREMENT PRIMARY KEY,
    pais_id VARCHAR(10)   NOT NULL,
    periodo_id INT        NOT NULL,
    anio INT              NOT NULL,
    unidad_id INT         NOT NULL,
    valor FLOAT,
    FOREIGN KEY (pais_id)     REFERENCES Paises(codigo),
    FOREIGN KEY (periodo_id)  REFERENCES Periodos(id),
    FOREIGN KEY (unidad_id)   REFERENCES Unidades(id)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
"""
cursor.execute(sql_create_emisiones)
conexion.commit()

# Insertar la unidad 'mt' si no existe
cursor.execute("""
  INSERT IGNORE INTO Unidades (descripcion)
  VALUES ('mt');
""")
conexion.commit()

# Recuperar el ID de la unidad 'mt'
cursor.execute("SELECT id FROM Unidades WHERE descripcion = 'mt';")
unit_id_mt = cursor.fetchone()[0]
print("unit_id para 'mt':", unit_id_mt)


unit_id para 'mt': 2


## 5. Insertar los datos de emisiones de CO₂

Cada fila de `df_co2` ya tiene su `pais_id` mapeado y el año. Dado que los datos son anuales, usaremos siempre `periodo_id = 17`. A continuación preparamos los *batch inserts* y volcamos los registros en la base de datos.


In [10]:
fact_emisiones = []
for _, row in df_co2.iterrows():
    fact_emisiones.append((
        row['pais_id'],
        17,
        int(row['Year']),
        unit_id_mt,
        row['co2_mt']
    ))

sql_insert_emisiones = """
INSERT INTO EmisionesCO2
  (pais_id, periodo_id, anio, unidad_id, valor)
VALUES (%s, %s, %s, %s, %s);
"""

# Parámetros de batching
batch_size = 1000
total = len(fact_emisiones)
print(f"Total a insertar: {total} filas")

for start in range(0, total, batch_size):
    end = start + batch_size
    batch = fact_emisiones[start:end]
    try:
        cursor.executemany(sql_insert_emisiones, batch)
        conexion.commit()
        print(f"  ✔ Insertadas filas {start + 1}–{min(end, total)}")
    except pymysql.err.OperationalError as e:
        print(f"  ❌ Error al insertar lote {start + 1}–{min(end, total)}:", e)
        conexion.rollback()
        break

# Clean up
cursor.close()
conexion.close()


Total a insertar: 5422 filas
  ✔ Insertadas filas 1–1000
  ✔ Insertadas filas 1001–2000
  ✔ Insertadas filas 2001–3000
  ✔ Insertadas filas 3001–4000
  ✔ Insertadas filas 4001–5000
  ✔ Insertadas filas 5001–5422
